# Unagged cross-section ratio of $d(e,e'\pi^+)/d(e,e'\pi^+)$ 


last edit Sep-26, 2023

## Imports and definitions

In [1]:
import sys; 
software_path = '/Users/erezcohen/Desktop/Software/'
sys.path.insert(0, software_path + '/mySoftware/Python/');
sys.path.insert(0, software_path + '/CLAS12/BAND/SIDIS_at_BAND/PythonAnalysis/AcceptanceCorrections/');
sys.path.insert(0, software_path + '/CLAS12/BAND/SIDIS_at_BAND/PythonAnalysis/python_auxiliary/');
from my_tools                     import *; 
from plot_tools                   import *;
from my_data_analysis_tools       import *;
from acceptance_correction_tools  import *;
from sidis_analysis_tools         import *;

Welcome to JupyROOT 6.26/04


/Users/erezcohen/Desktop/Software//CLAS12/BAND/SIDIS_at_BAND/PythonAnalysis/python_auxiliary/sidis_analysis_tools.py:62: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  weight_per_run_rga[run] = 1./float(beam_charge_all_runs_rga[beam_charge_all_runs_rga.runnum==run].beam_charge);


Loaded bin migration and acceptance weights from MC calculations.


In [2]:
%config InlineBackend.figure_format = 'retina'
plt.rcParams['mathtext.fontset']    = 'stix'
plt.rcParams['font.family']         = 'STIXGeneral'

## (1) Load data

In [3]:
subdirname = "sidisdvcs_24Mar2023_commit_cfbc431" 
e_e_pi,_,_ = load_SIDIS_data( Nruns = -1,
                             rgb_runs_filenames = ["good_runs_10-2-final.txt",
                                                   "good_runs_10-4.txt",
                                                   "good_runs_10-6.txt"], 
                             subdirname = subdirname,
                             do_e_e_pi_n=False, do_e_e_pi_FreeP=False, 
                             fdebug=0 );

Done loading files.


## (2) Apply selection cuts not previously imposed

In [4]:
import time

In [5]:
print('number of pi+ events: %.1f'%(len(e_e_pi['piplus'])/1e6),'M')

number of pi+ events: 60.3 M


In [6]:
# AcceptanceMatchingType  ='p-theta-phi'
AcceptanceMatchingType  ='p-theta'
Nevents = -1
t0 = time.time()
e_e_pi_pass_cuts,_,_,_ = apply_further_selection_cuts_to_data(fdebug=0, 
                                                              NeventsMax=Nevents,
                                                              AcceptanceMatchingType = AcceptanceMatchingType);
t1 = time.time() 
print("Time elapsed: %.2f"%(t1 - t0), 'sec (%g us/event)'%((t0-t1)/Nevents*1.e6)) # CPU seconds elapsed (floating point)

Applying selection cuts not previously imposed
Apply a π+/π- acceptance matching cut on the in p-theta plane


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
Nevents = -1
t0= time.time()
e_e_pi_pass_cuts_3DAcceptanceMatch,_,_,_ = apply_further_selection_cuts_to_data(fdebug=0, 
                                                                                NeventsMax=Nevents,
                                                                                AcceptanceMatchingType  ='p-theta-phi');
t1 = time.time() 
print("Time elapsed: %.2f"%(t1 - t0), 'sec (%g us/event)'%((t0-t1)/Nevents*1.e6)) # CPU seconds elapsed (floating point)

## (3) Define binning for cross-section ratio as a function of Bjorken $x$, and results path
For a standing proton use $x_B$, defined as
$ x_B = Q^2/2m_p\omega$

In [ ]:
z_bins   = np.arange(0.3,0.8,0.05)
z_widths = 0.01*np.ones(len(z_bins))
x_bins   = np.linspace(0.22,0.62,11)
x        = (x_bins[1:] + x_bins[:-1])/2
x_err    = (x_bins[1:] - x_bins[:-1])/2
print('x bins:',x_bins,',\nz bins:',z_bins)
for z_bin,z_width in zip(z_bins,z_widths):
    z_min,z_max = z_bin-z_width, z_bin+z_width
    print('%.3f < z < %.3f:'%(z_min,z_max))

In [ ]:
var_label = "standing proton x"
var    = 'xB'
xlabel = "Bjorken $x$"

In [ ]:
results_path = '/Users/erezcohen/Desktop/data/BAND/Results/' + 'Results_10June2023/'

# (4) Extract cross-section ratio as a function of $x_B$ and save tabular data to results files

## (4.0) Integrated on kinematics

In [ ]:
prefix = 'Untagged_SIDIS_ratio_'
suffix = ''

In [ ]:
data_path = results_path + 'IntegratedOnKinematics/'
if not os.path.exists(data_path): os.makedirs(data_path)
extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts,                           
                    x_var    = var ,                    
                    data_path=data_path,                    
                    x_bins   = x_bins,                                               
                    z_bins   = z_bins,                                               
                    z_widths = z_widths,                                               
                    fdebug   = 0,                    
                    W_min    = 2.5,                    
                    prefix   = prefix,
                    suffix   = suffix,
                         # weight_option='bin migration + acceptance + meson subtraction'
                         weight_option='bin migration + acceptance'
                        )

In [ ]:
SIDIS_results = load_SIDIS_ratio(prefix = prefix, 
                                 suffix = suffix, 
                                 doPlotResults=True, 
                                 data_path=data_path,fdebug=3)
ax = plt.gca()
ax.set_ylim(1,2.5);

## (4.1) Results in different bins of $Q^2$ integrated over $W$

In [ ]:
data_path= results_path + 'Q2_bins/'
if not os.path.exists(data_path): os.makedirs(data_path)
Q2_min_arr = np.arange(2,8,0.5)
Q2_max_arr = Q2_min_arr + 0.5

In [ ]:
for Q2_min,Q2_max,indcs in zip(Q2_min_arr,Q2_max_arr,range(10)):
    suffix   = '_%.1f_Q2_%.1f'%(Q2_min,Q2_max)
    print(suffix)
    extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts, 
                        x_var    = var ,
                        x_bins   = x_bins,
                        z_bins   = z_bins,
                        z_widths = z_widths,          
                        fdebug   = 0,
                        data_path= data_path,
                        prefix   = 'Untagged_SIDIS_ratio_',                    
                        suffix   = suffix,
                        Q2_min   = Q2_min,
                        Q2_max   = Q2_max, 
                             # weight_option='bin migration + acceptance + meson subtraction'
                             weight_option='bin migration + acceptance'
                       )
print('done.')

In [ ]:
data_path= results_path + 'Q2_bins_with_partial_corrections/'
if not os.path.exists(data_path): os.makedirs(data_path)

for Q2_min,Q2_max in zip([2,3,4],[2.5,3.5,4]):
    for suffix_str,weight_option in zip(['meson_subtraction','bin_migration','acceptance','bin_migration_acceptance','bin_migration_acceptance_meson_subtraction'],                                   
                                        ['meson subtraction','bin migration','acceptance','bin migration + acceptance','bin migration + acceptance + meson subtraction']):
        suffix   = suffix_str +'_%.1f_Q2_%.1f'%(Q2_min,Q2_max)
        print(suffix)
        extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts, 
                            x_var    = var ,
                            x_bins   = x_bins,
                            z_bins   = z_bins,
                            z_widths = z_widths,          
                            fdebug   = 0,
                            data_path= data_path,
                            prefix   = 'Untagged_SIDIS_ratio_',                    
                            suffix   = suffix,
                            Q2_min   = Q2_min,
                            Q2_max   = Q2_max, 
                                 weight_option=weight_option
                           )
print('done.')

## (4.2) $Q^2$ bins integrated over $W$, compare accetpance matching in $p-\theta$ vs. $p-\theta-\phi$

In [ ]:
data_path = results_path + 'Q2_bins_AcceptanceMatch3D/'
if not os.path.exists(data_path): os.makedirs(data_path)

In [ ]:
for Q2_min,Q2_max in zip(Q2_min_arr,Q2_max_arr):
    suffix   = '_%.1f_Q2_%.1f'%(Q2_min,Q2_max)
    print(suffix)
    extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts_3DAcceptanceMatch, 
                        x_var    = var ,
                        x_bins   = x_bins,
                        z_bins   = z_bins,
                        z_widths = z_widths,          
                        fdebug   = 0,
                        data_path= data_path,
                        prefix   = 'Untagged_SIDIS_ratio_',                    
                        suffix   = suffix,
                        Q2_min   = Q2_min,
                        Q2_max   = Q2_max,
                        weight_option='bin migration + acceptance'
                       )
print('done.')

# (5) Extract modified results
E.g. in different kinematical bins 

## (5.1) Difference between different beam eneries

Since there are slightly different kinematics - due to the different beam enegries,
we match the $Q^2$ range for all energies to $3 < Q^2 < 4$ (GeV/c)$^2$

In [ ]:
data_path = results_path + 'DifferentBeamEnergies/'
if not os.path.exists(data_path): os.makedirs(data_path)
Q2_min = 4.0
Q2_max = 4.5

In [ ]:
indcs = dict()
e_e_pi_pass_cuts_DifferentEbeam = dict()
for E, rum_min, run_max in zip([10.2,10.4 ,10.6],
                               [6400,11360,6160],
                               [6600,11570,6400]):
    e_e_pi_pass_cuts_DifferentEbeam[E] = dict()
    for pi_ch in pi_charge_names:
        indcs[pi_ch + '%.1f'%E] = np.where( (rum_min < e_e_pi_pass_cuts[pi_ch].runnum)                               
                                           & 
                                           (e_e_pi_pass_cuts[pi_ch].runnum < run_max)) 
        e_e_pi_pass_cuts_DifferentEbeam[E][pi_ch] = e_e_pi_pass_cuts[pi_ch].iloc[indcs[pi_ch + '%.1f'%E]]
        
print('Done')        

In [ ]:
for E, rum_min, run_max in zip([10.2,10.4 ,10.6],
                               [6400,11360,6160],
                               [6600,11570,6400]):

    extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts_DifferentEbeam[E],
                        x_var    = var ,
                        x_bins   = x_bins,
                        z_bins   = z_bins,
                        z_widths = z_widths,          
                        fdebug   = 0,
                        data_path= data_path,
                        prefix   = 'Untagged_SIDIS_ratio_',                    
                        suffix   = '_E_%.1fGeV_%.1f_Q2_%.1f_GeV'%(E,Q2_min,Q2_max),
                        Q2_min   = Q2_min,
                        Q2_max   = Q2_max)
print('Done extracting cross-sections for different E(beam)')

## (5.2) Different electron sector

In [ ]:
data_path = results_path + 'DifferentElectronSector/'
if not os.path.exists(data_path): os.makedirs(data_path)
Q2_min,Q2_max = 4.0, 4.5

In [ ]:
indcs = dict()
e_e_pi_pass_cuts_Different_eSector = dict()
for eSector in np.arange(1,7,1):
    e_e_pi_pass_cuts_Different_eSector[eSector] = dict()
    for pi_ch in pi_charge_names:
        indcs[pi_ch + '%d'%eSector] = np.where( e_e_pi_pass_cuts[pi_ch].e_DC_sector==eSector ) 
        e_e_pi_pass_cuts_Different_eSector[eSector][pi_ch] = e_e_pi_pass_cuts[pi_ch].iloc[indcs[pi_ch + '%d'%eSector]]
        
print('Done')        

In [ ]:
for eSector in np.arange(1,7,1):

    extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts_Different_eSector[eSector],
                        x_var    = var ,
                        x_bins   = x_bins,
                        z_bins   = z_bins,
                        z_widths = z_widths,          
                        fdebug   = 0,
                        data_path= data_path,
                        prefix   = 'Untagged_SIDIS_ratio_',                    
                        suffix   = '_e_Sector_%d_%.1f_Q2_%.1f_GeV'%(eSector,Q2_min,Q2_max),
                        Q2_min   = Q2_min,
                        Q2_max   = Q2_max)
print('Done extracting cross-sections for different e-Sector')

## (5.3) Different pion sector

In [ ]:
data_path = results_path + 'DifferentPiSector/'
if not os.path.exists(data_path): os.makedirs(data_path)
Q2_min,Q2_max = 4.0, 4.5

In [ ]:
indcs = dict()
e_e_pi_pass_cuts_Different_PiSector = dict()
for PiSector in np.arange(1,7,1):
    e_e_pi_pass_cuts_Different_PiSector[PiSector] = dict()
    for pi_ch in pi_charge_names:
        indcs[pi_ch + '%d'%PiSector] = np.where( e_e_pi_pass_cuts[pi_ch].pi_DC_sector==PiSector ) 
        e_e_pi_pass_cuts_Different_PiSector[PiSector][pi_ch] = e_e_pi_pass_cuts[pi_ch].iloc[indcs[pi_ch + '%d'%PiSector]]
        
print('Done')        

In [ ]:
for PiSector in np.arange(1,7,1):
    extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts_Different_PiSector[PiSector],
                        x_var    = var ,
                        x_bins   = x_bins,
                        z_bins   = z_bins,
                        z_widths = z_widths,          
                        fdebug   = 0,
                        data_path= data_path,
                        prefix   = 'Untagged_SIDIS_ratio_',                    
                        suffix   = '_Pi_Sector_%d_%.1f_Q2_%.1f_GeV'%(PiSector,Q2_min,Q2_max),
                        Q2_min   = Q2_min,
                        Q2_max   = Q2_max)
print('Done extracting cross-sections for different π-Sector')

## (5.4) Different bins of $M_x$

In [ ]:
data_path= results_path + 'Mx_bins/'
if not os.path.exists(data_path): os.makedirs(data_path)
M_x_min_arr = [1.7,2.0,2.5]
M_x_max_arr = [2.5,3.0,3.5]

In [ ]:
for M_x_min,M_x_max in zip(M_x_min_arr,M_x_max_arr):
    print(M_x_min,'<M_x<',M_x_max)
    extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts,
                        x_var    = var ,
                        x_bins   = x_bins,
                        z_bins   = z_bins,
                        z_widths = z_widths,          
                        fdebug   = 0,
                        data_path= data_path,
                        prefix   = 'Untagged_SIDIS_ratio_',                    
                        suffix   = '_Mx_%.2f-%.2f_GeV'%(M_x_min,M_x_max),
                        M_x_min  = M_x_min,
                        M_x_max  = M_x_max)
    
print('done.')

### (5.4.1) bins of $M_x$ and $Q^2$

In [ ]:
data_path= results_path + 'Mx_and_Q2_bins/'
if not os.path.exists(data_path): os.makedirs(data_path)
M_x_min_arr = [0.0,2.0]
M_x_max_arr = [2.0,4.0]
Q2_min_arr  = [2.0,4.0]
Q2_max_arr  = [4.0,9.0]


for Q2_min,Q2_max in zip(Q2_min_arr,Q2_max_arr):
    print(Q2_min,'<Q2<',Q2_max)

    for M_x_min,M_x_max in zip(M_x_min_arr,M_x_max_arr):
        print('\t',M_x_min,'<M_x<',M_x_max)

        extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts,
                            x_var    = var ,
                            x_bins   = x_bins,
                            z_bins   = z_bins,
                            z_widths = z_widths,          
                            fdebug   = 0,
                            data_path= data_path,
                            prefix   = 'Untagged_SIDIS_ratio_',                    
                            suffix   = '_Mx_%.2f-%.2f_GeV_Q2_%.2f-%.2f_GeV'%(M_x_min,M_x_max,Q2_min,Q2_max),
                            Q2_min   = Q2_min,
                            Q2_max   = Q2_max,
                            M_x_min  = M_x_min,
                            M_x_max  = M_x_max)
    
print('done.')

## (5.5) Results at high-$Q^2$ in different bins of $p_T$ or $\phi_\pi$

### (5.5.1) depepence on $p_T$

In [ ]:
# Q2_min,Q2_max = 4.0,4.5
# Q2_min,Q2_max = 3.0,3.5
# Q2_min,Q2_max = 2.0,2.5

In [ ]:
dpT = 0.25
pT_min_arr = np.arange(0,1,dpT)
pT_max_arr = pT_min_arr + dpT
pT_min_arr, pT_max_arr

In [ ]:
for Q2min,Q2max in zip([2.,3.,4.],[2.5,3.5,4.5]):
    data_path= results_path + 'pT_bins_%.1f_Q2_%.1f/'%(Q2min,Q2max)
    if not os.path.exists(data_path): os.makedirs(data_path)

    for pT_min,pT_max in zip(pT_min_arr,pT_max_arr):
        suffix   = '_%.2f_pT_%.2f'%(pT_min,pT_max)
        print(suffix)
        extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts, 
                            x_var    = var ,
                            x_bins   = x_bins,
                            z_bins   = z_bins,
                            z_widths = z_widths,          
                            fdebug   = 0,
                            data_path= data_path,
                            prefix   = 'Untagged_SIDIS_ratio_',                    
                            suffix   = suffix,
                            Q2_min   = Q2min,
                            Q2_max   = Q2max,
                            pT_min   = pT_min,
                            pT_max   = pT_max,
                           )
    print('Done %.1f<Q2<%.1f'%(Q2min,Q2max))
print('done.')

### (5.5.2) depepence on $\phi$
First look a the coupling between $\phi$ and $p_T$

In [ ]:
for pi_ch,pi_idx in zip(pi_charge_,[1,2]):
    e_e_pi_pass_cuts[pi_ch]

In [ ]:
phi_min_arr = np.arange(-np.pi,np.pi-np.pi/3,np.pi/3)
phi_max_arr = phi_min_arr + np.pi/3
phi_min_arr, phi_max_arr

In [ ]:
for Q2min,Q2max in zip([2.,3.,4.],[2.5,3.5,4.5]):
    data_path= results_path + 'phi_bins_%.1f_Q2_%.1f/'%(Q2min,Q2max)
    if not os.path.exists(data_path): os.makedirs(data_path)

    for phi_min,phi_max in zip(phi_min_arr, phi_max_arr):
        suffix   = '_%.2f_phi_%.2f'%(phi_min,phi_max)
        print(suffix)
        extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts, 
                            x_var    = var ,
                            x_bins   = x_bins,
                            z_bins   = z_bins,
                            z_widths = z_widths,          
                            fdebug   = 0,
                            data_path= data_path,
                            prefix   = 'Untagged_SIDIS_ratio_',                    
                            suffix   = suffix,
                            Q2_min   = Q2min,
                            Q2_max   = Q2max,
                            phi_min   = phi_min,
                            phi_max   = phi_max,
                           )
    print('Done %.1f<Q2<%.1f'%(Q2min,Q2max))        
print('done.')

## (5.6) Different bins of $Q^2$ and $W$

In [ ]:
# from IPython.display import IFrame
# IFrame('Q2.png', width=600, height=300)

In [ ]:
# IFrame('W.png', width=600, height=300)

In [ ]:
Q2_min_arr = [2.0, 2.8, 3.6]
Q2_max_arr = [2.8, 3.6, 8.0]

W_min_arr = np.linspace(2.5,3.1,3)
W_max_arr = np.linspace(2.8,3.4,3)
# print(W_min_arr)  # print(W_max_arr)

z_bins   = np.arange(0.3,0.8,0.05)
z_widths = 0.01*np.ones(len(z_bins))

In [ ]:
data_path= '/Users/erezcohen/Desktop/data/BAND/Results/Q2_and_W_bins/'
if not os.path.exists(data_path): os.makedirs(data_path)


for Q2_min,Q2_max in zip(Q2_min_arr,Q2_max_arr):
    for W_min,W_max in zip(W_min_arr,W_max_arr):
        suffix   = '_%.1f_Q2_%.1f____%.1f_W_%.1f'%(Q2_min,Q2_max,W_min,W_max)
        print(suffix)
        extract_SIDIS_Xsec_ratio(df_dict  = e_e_pi_pass_cuts, 
                            x_var    = var ,
                            x_bins   = x_bins,
                            z_bins   = z_bins,
                            z_widths = z_widths,          
                            fdebug   = 0,
                            data_path= data_path,
                            prefix   = 'Untagged_SIDIS_ratio_',                    
                            suffix   = suffix,
                            W_min    = W_min,
                            W_max    = W_max,
                            Q2_min   = Q2_min,
                            Q2_max   = Q2_max
                           )

print('done.')

In [ ]:
NQ2 = len(Q2_min_arr)
NW  = len(W_min_arr)
fig = plt.figure(figsize=(16,16),tight_layout=True)
for Q2_min,Q2_max,Q2_idx in zip(Q2_min_arr,Q2_max_arr, range(NQ2)):
    for W_min,W_max,W_idx in zip(W_min_arr,W_max_arr, range(NW)):
        suffix   = '_%.1f_Q2_%.1f____%.1f_W_%.1f'%(Q2_min,Q2_max,W_min,W_max)

        axPlot = fig.add_subplot(NQ2, NW, Q2_idx*NW + W_idx + 1)
        SIDIS_results = load_SIDIS_ratio(prefix = prefix, 
                                         fdebug=3,
                                         axPlot = axPlot, 
                                         titlePlot = '$%.1f < Q^2 < %.1f$ (GeV/c)$^2$, $%.1f < W < %.1f$ GeV/c$^2$'%(Q2_min,Q2_max,W_min,W_max),
                                         data_path= data_path,
                                         suffix = suffix, 
                                         doPlotResults=True, 
                                         Nzbins2Plot=3)
        axPlot.set_xticks([0.2,0.3,0.4,0.5]);

## (5.7) Results for many more bins in $z$

In [ ]:
# z_bins   = np.arange(0.3,0.8,0.01)
# z_widths = 0.001*np.ones(len(z_bins))

# prefix = 'Untagged_SIDIS_ratio_'
# suffix = ''

In [ ]:
# extract_SIDIS_ratio(df_dict  = e_e_pi_pass_cuts,                                
#                            x_var    = var ,                           
#                            x_bins   = x_bins,                           
#                            z_bins   = z_bins,                           
#                            z_widths = z_widths,                           
#                            fdebug   = 0,
#                            prefix   = 'Untagged_SIDIS_ratio_',
#                             data_path= '/Users/erezcohen/Desktop/data/BAND/Results/Q2_and_W_bins/many_z_bins/',
#                            suffix   = '')

In [ ]:
# SIDIS_results = load_SIDIS_ratio_DataFrame(z_bins=z_bins,z_widths=z_widths,
#                                            prefix = prefix, suffix = suffix, 
#                                            doPlotResults=True)

# (6) Results with no acceptance matching cut in $p-\theta$ 

In [ ]:
Nevents = -1
t0= time.time()
e_e_pi_pass_cuts_no_p_theta_cut,_,_ = apply_further_selection_cuts_to_data(fdebug=2,
                                                                           NeventsMax=Nevents, 
                                                                           doAcceptanceMatchingCut=False);
t1 = time.time() 
print("Time elapsed: %.2f"%(t1 - t0), 'sec (%g us/event)'%((t1 - t0)/Nevents*1.e6)) # CPU seconds elapsed (floating point)

In [ ]:
save_SIDIS_ratio_DataFrame(df_dict  = e_e_pi_pass_cuts_no_p_theta_cut,                                
                           x_var    = var ,                           
                           x_bins   = x_bins,                           
                           z_bins   = z_bins,                           
                           z_widths = z_widths,                           
                           fdebug   = 0,
                           prefix   = 'Untagged_SIDIS_ratio_',                           
                           suffix   = '_No_AcceptanceMatchingCut')

In [ ]:
prefix = 'Untagged_SIDIS_ratio_'
suffix = '_No_AcceptanceMatchingCut'
SIDIS_results = load_SIDIS_ratio_DataFrame(prefix = prefix, suffix = suffix, doPlotResults=True)

# arxiv

## Check the impact of beam charge normalization on the data

In [ ]:
e_e_pi_pass_cuts['piplus'].runnum

In [ ]:

# Rpips2pims, Rpips2pims_errup, Rpips2pims_errdw = dict(), dict(), dict()
# Y, dY = dict(),dict()
# for z_bin,z_width in zip(z_bins,z_widths):
#     z_min,z_max = z_bin-z_width,z_bin+z_width
#     (Rpips2pims[z_bin],
#      Rpips2pims_errup[z_bin], 
#      Rpips2pims_errdw[z_bin],_,_) = compute_ratio_pips_to_pims(df_dict=e_e_pi_pass_cuts  , 
#                                                              var=var, 
#                                                              bins=x_bins, 
#                                                              z_min=z_min,z_max=z_max)

#     # fit to a constant 
#     y    = Rpips2pims[z_bin]
#     y_err= (Rpips2pims_errup[z_bin],Rpips2pims_errdw[z_bin])
#     p,cov = np.polyfit(x,y,deg=0,cov=True)
#     Y[z_bin],dY[z_bin] = p[0],cov[0,0]

#     # print(z_bin,Rpips2pims_n[z_bin],Rpips2pims_n_errup[z_bin],Rpips2pims_n_errdw[z_bin])
# print('done') 

In [ ]:
# fig = plt.figure(figsize=(9,6))
# ax  = fig.add_subplot(1,1,1)
# for z_bin,z_width in zip(z_bins,z_widths):
#     y    = Rpips2pims[z_bin]
#     y_err= (Rpips2pims_errup[z_bin],Rpips2pims_errdw[z_bin])
#     x_fit = np.linspace(np.min(x),np.max(x),100)
#     y_dw = (Y[z_bin]-dY[z_bin])*np.ones(len(x_fit))
#     y_up = (Y[z_bin]+dY[z_bin])*np.ones(len(x_fit))

#     # plot
#     l=ax.errorbar(x=x, xerr=x_err,  y=y, yerr=y_err,
#                 marker='o',markeredgecolor='k',
#                 label='$z=%.2f\pm%.2f, (\pi^+/\pi^-)=%.2f\pm%.2f$'%(z_bin,z_width,Y[z_bin],dY[z_bin]))
    
#     ax.fill_between( x_fit, y_dw, y_up, color=l[0].get_color(), alpha=0.1)
# set_axes(ax,xlabel,"$N(e,e'\pi^+)/N(e,e'\pi^-)$",
#          title="$\pi^+/\pi^-$ ratio as a function of $x_B$ without a tagged neutron",
#          do_add_grid=True, do_add_legend=True, fontsize=18,
#         );
# plt.legend(bbox_to_anchor=(1,1.05),loc='best',fontsize=18)

### Improve analysis timing with a focus on apply_further_selection_cuts_to_data()
Check timing of the function and improve it, as well as memory usage

In [ ]:
import timeit
timeit.timeit("apply_further_selection_cuts_to_data(fdebug=2, NeventsMax=-1)", 
              setup="from __main__ import apply_further_selection_cuts_to_data",number=1)

In [ ]:
e_e_pi[pi_ch].keys()
# retain only the most important features to reduce memory usage by a large pandas DataFrame
reduced_e_e_pi = dict()
for pi_ch in pi_charge_names:
    reduced_e_e_pi[pi_ch] = e_e_pi[pi_ch][['runnum','evnum',
                                           'e_P','e_Theta','e_Phi', 
                                           'pi_P', 'pi_Theta', 'pi_Phi', 
                                           'Q2', 'W', 'xB', 'Zpi',
                                           'M_X', 'e_DC_sector', 'pi_DC_sector']]
    reduced_e_e_pi[pi_ch] = reduced_e_e_pi[pi_ch].astype({"evnum": int,
                                                          "e_P":np.half,"e_Theta":np.half,"e_Phi":np.half,
                                                          "pi_P":np.half,'pi_Theta':np.half, 'pi_Phi':np.half,
                                                          'Q2':np.half, 'W':np.half, 'xB':np.half, 'Zpi':np.half,
                                                          'M_X':np.half, })

reduced_e_e_pi[pi_ch][0:5]

In [ ]:
for pi_ch in pi_charge_names:
    print(pi_ch)
    print('before cuts')
    print(e_e_pi[pi_ch].info(memory_usage="deep",verbose=False))
    print(' ')    
    # print('reduced dataframe before cuts')
    # print(reduced_e_e_pi[pi_ch].info(memory_usage="deep",verbose=False))
    # print(' ')
    print('after cuts')
    print(e_e_pi_pass_cuts[pi_ch].info(memory_usage="deep",verbose=False))
    print(' ')

In [ ]:
e_e_pi_pass_cuts[pi_ch].index